In [1]:
import torch
import model

ModuleNotFoundError: No module named 'model'

In [2]:
model = model.MiniConvNetFull()
model.load_state_dict(torch.load("model.pkl", map_location='cpu'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [3]:
for name,parameters in model.named_parameters():
    print(name,':',parameters.size())

features.0.weight : torch.Size([32, 1, 3, 3])
features.1.weight : torch.Size([32])
features.1.bias : torch.Size([32])
features.4.weight : torch.Size([32, 32, 3, 3])
features.5.weight : torch.Size([32])
features.5.bias : torch.Size([32])
features.7.weight : torch.Size([32, 32, 3, 3])
features.8.weight : torch.Size([32])
features.8.bias : torch.Size([32])
features.11.weight : torch.Size([32, 32, 3, 3])
features.12.weight : torch.Size([32])
features.12.bias : torch.Size([32])
classifier.0.weight : torch.Size([20, 1568])
classifier.1.weight : torch.Size([20])
classifier.1.bias : torch.Size([20])
classifier.3.weight : torch.Size([10, 20])


In [4]:
for name, sub_module in model.named_modules():
    if type(sub_module).__base__ is torch.nn.Conv2d:
        print(name)
        print(sub_module.bias)

# 生成配置头文件

In [5]:
def write_define(define_name, define_value, config_file):
    config_file.write("#define "+str(define_name)+" "+str(define_value)+"\n")
    
def generateConfig(model, in_row, in_col, path_to_config):
    config_file = open(path_to_config, "w")
    cov_cnt = 0
    pool_cnt = 0
    linear_cnt = 0
    in_ch = 0
    for name, sub_module in model.named_modules():
        if type(sub_module) is torch.nn.Conv2d:
            write_define("COV_" + str(cov_cnt) + "_IN_CH", sub_module.in_channels, config_file)
            write_define("COV_" + str(cov_cnt) + "_IN_ROW", in_row, config_file)
            write_define("COV_" + str(cov_cnt) + "_IN_COL", in_col, config_file)
            
            in_row = (in_row + 2 * sub_module.padding[0] - sub_module.kernel_size[0]) // sub_module.stride[0] + 1
            in_col = (in_col + 2 * sub_module.padding[0] - sub_module.kernel_size[0]) // sub_module.stride[0] + 1
            in_ch = sub_module.out_channels
            write_define("COV_" + str(cov_cnt) + "_OUT_CH", sub_module.out_channels, config_file)
            write_define("COV_" + str(cov_cnt) + "_OUT_ROW", in_row, config_file)
            write_define("COV_" + str(cov_cnt) + "_OUT_COL", in_col, config_file)
            
           
            write_define("COV_" + str(cov_cnt) + "_K", sub_module.kernel_size[0], config_file)
            write_define("COV_" + str(cov_cnt) + "_S", sub_module.stride[0], config_file)
            write_define("COV_" + str(cov_cnt) + "_P", sub_module.padding[0], config_file)
            if sub_module.bias != None:
                write_define("COV_" + str(cov_cnt) + "_B", 1, config_file)
            else:
                write_define("COV_" + str(cov_cnt) + "_B", 0, config_file)
            cov_cnt = cov_cnt + 1
            config_file.write("\n")
        elif type(sub_module) is torch.nn.MaxPool2d:
            write_define("POOL_" + str(pool_cnt) + "_IN_CH", in_ch, config_file)
            write_define("POOL_" + str(pool_cnt) + "_IN_ROW", in_row, config_file)
            write_define("POOL_" + str(pool_cnt) + "_IN_COL", in_col, config_file)
            write_define("POOL_" + str(pool_cnt) + "_IN_PO", sub_module.kernel_size, config_file)
            in_row = in_row // sub_module.kernel_size
            in_col = in_col // sub_module.kernel_size
            pool_cnt = pool_cnt + 1
            config_file.write("\n")
        elif type(sub_module) is torch.nn.Linear:
            write_define("LINEAR_" + str(linear_cnt) + "_IN_N", sub_module.in_features, config_file)
            write_define("LINEAR_" + str(linear_cnt) + "_OUT_N", sub_module.out_features, config_file)
            linear_cnt = linear_cnt + 1
            config_file.write("\n")
#         elif type(sub_module) is torch.nn.ReLU:
#             write_define("LINEAR_" + str(linear_cnt) + "_IN_N", sub_module.in_features, config_file)
    config_file.close()

In [6]:
generateConfig(model, 28, 28, 'config.h')

# 生成参数

In [41]:
import numpy as np

def generate_params(model):
    
    cov_cnt = 0
    linear_cnt = 0
    bn_cnt = 0

    for name, sub_module in model.named_modules():
        if type(sub_module) is torch.nn.Conv2d:
            w = sub_module.weight.detach().numpy()
#             print(w)
            w.tofile('prama/' + 'conv_' + str(cov_cnt) + '_w.bin')
            if sub_module.bias != None:
                b = sub_module.bias.detach().numpy()
                b.tofile('prama/' + 'conv_' + str(cov_cnt) + '_b.bin')
            cov_cnt = cov_cnt + 1    
        elif type(sub_module) is torch.nn.Linear:
            w = sub_module.weight.detach().numpy()
            w.tofile('prama/' + 'linear_' + str(linear_cnt) + '_w.bin')
            if sub_module.bias != None:
                b = sub_module.bias.detach().numpy()
                b.tofile('prama/' + 'linear_' + str(linear_cnt) + '_b.bin')          
            linear_cnt = linear_cnt + 1
        elif type(sub_module) is torch.nn.BatchNorm2d or type(sub_module) is torch.nn.BatchNorm1d:
            gamma = sub_module.weight.detach().numpy()
            var = sub_module.running_var.numpy()
            mean = sub_module.running_mean.numpy()
            eps = sub_module.eps
            beta = sub_module.bias.detach().numpy()
            
#             print("gamma ", gamma)
#             print("var", var)
#             print('mean', mean)
#             print('beta', beta)
            
            
            w = gamma / (np.sqrt(var) + eps)
            w.tofile('prama/' + 'bn_' + str(bn_cnt) + '_w.bin')

            
            b = beta - (mean / (np.sqrt(var) + eps) * gamma)
            print(eps)
            print('beta = ', beta)
            print('b = ', b)
            b.tofile('prama/' + 'bn_' + str(bn_cnt) + '_b.bin') 
            bn_cnt = bn_cnt + 1

generate_params(model)

1e-05
beta =  [ 0.02388065  0.01089465  0.01564483  0.05289611  0.01513779 -0.02877514
 -0.0248562   0.00178388  0.00945559  0.00549961  0.02283658  0.05723793
  0.01010213  0.00596863 -0.00542759  0.0146016  -0.02077629 -0.00387304
  0.03725917 -0.01456535  0.00192923  0.01053245 -0.02995951  0.00337649
  0.0514285  -0.01337652  0.03363698  0.00642971  0.05298251  0.0140421
  0.00966313  0.03996097]
b =  [ 0.23054919  0.20878126 -0.39858305  0.374077    0.4325052   0.06877299
  0.04340973  0.00286376 -0.25673512  0.13347222  0.3056556   0.01454846
 -0.18620497 -0.18566419  0.00149324 -0.09152427 -0.04192213 -0.00331088
  0.0085146  -0.02244141 -0.06747825  0.24063674  0.10392877 -0.00134065
  0.1191881  -0.37271008  0.4595673   0.31261703  0.06125054 -0.38794965
 -0.23446953  0.17058368]
1e-05
beta =  [ 0.00142689  0.00931651 -0.02187369 -0.01334099  0.00024301  0.00831488
 -0.04687087 -0.01488988 -0.04253968  0.01269346  0.00855247 -0.02022107
 -0.02290761  0.01957975 -0.02408206 -0.

In [18]:
a = np.array([1, 2, 3, 4])
a = a + 1.22
print(a)

[2.22 3.22 4.22 5.22]
